# `hv.Holomaps` with FLDAS dataset
Modified: Jun 2, 2019

Eastern & Southern Africa fldas data

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint, dict2json, display_dict2json, is_valid_url
import utils



### Use holoview's colormaps module
<img src="../assets/hv_colormaps.png" alt="holoviews-colormaps" width="600"/>

In [ ]:
from holoviews.plotting import list_cmaps

In [ ]:
# list_cmaps(provider='colorcet', category='Sequential')

In [ ]:
%opts Image [colorbar=True, tools=['hover']] Curve [tools=['hover']]

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())

In [ ]:
renderer = hv.renderer('bokeh')

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
fpath_sa = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
fpath_ea = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

#set height, width for images (for plotting)
H_IMG, W_IMG = 400, 400

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)



In [ ]:
# Simple example of `hv.dim()` funnction to transform dimensions
# Useful to transform a dimension before mapping it to a visual attribute for plotting custromization
# src: http://holoviews.org/user_guide/Style_Mapping.html
hv.dim('X').apply(hvd_sa)
hv.dim('time').apply(hvd_sa)

In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = hv.Image(xrd_ea[varname].isel(time=3) , ['X','Y']) #Opt: vdims=varname
print(timg_ea)

timg_ea.opts(width=W_IMG, height=H_IMG)

In [ ]:
# Now let's see how we can use holoview's plotting.opts function with `cmap` and `color_level` keywords to 
# categorize the values to certain number of color_levels easily

( 
    timg_ea.relabel('default, continuous level cmapping').opts(cmap='PiYG') + \
    timg_ea.relabel('5 color levels').opts(cmap='PiYG', color_levels=5) + \
    timg_ea.relabel('11 color levels').opts(cmap='PiYG', color_levels=11)
).opts(width=300,height=300).cols(3)




Wow, this is really powerful. I love that I didn't need to categorize the values to 5 or 11 levels myself -> no new data variable for the result, so memory efficient too.

## cycle colormaps
Modified: Jun 7, 2019

In [ ]:
gdf

In [ ]:
# gdf
# gdf = get_vectile_gdf
# lat, lon, z = -5, 37, 9
# print(utils.get_latlon('paris, france'))
z = 16
lat, lon = utils.get_latlon('paris, france')
      

In [ ]:
gdf = get_vectile_gdf_at_latlon(lat, lon, z)
print(len(gdf))
display(gdf.head())

In [ ]:
addr = input('Enter query address: ')
lat, lon = utils.get_latlon(addr)
print('lat, lon: ', lat, lon)
z = int(input('Enter zoom level [0,16]:'))
shapes = get_vectile_overlay_at_latlon(lat,lon, z)
shapes

# try:
# addr: national museum of scotland, edinburgh, scotland
# lat, lon:  55.9470714 -3.18933554234473


In [ ]:
shapes = get_vectile_overlay_at_latlon(lat,lon, 12)


In [ ]:
shapes

In [ ]:
# hv.DynamicMap(get_vectile_overlay_at_latlon, kdims=['lat', 'lon', 'z']).
# redim.values(lat=[

## A list of Lat, Lon data for world cities
Modified: Jun 7, 2019
- Plot a table of word citiy location data
and link the table with a DynamicMap that shows the vector tiles at that location 
(with default zoom of 14(?) 


In [ ]:
latlons = pd.read_csv('/home/hayley/data/latlon/worldcities.csv')
latlons = latlons[['city_ascii', 'lat', 'lng', 'country']]

In [ ]:
sample_latlons = latlons.sample(frac=0.01)
len(sample_latlons)

In [ ]:
sample_latlons = (
    sample_latlons
    .rename({'city_ascii':'city'}, axis='columns')
    .sort_values(by='country')
#     .reset_index()
)

In [ ]:
latlon_table = hv.Table(sample_latlons, 
                        kdims = ['country', 'city', 'lat', 'lng'],
                        label='city_latlon').opts(selectable=True)

In [ ]:
hv.extension('bokeh')

In [ ]:
latlon_table#.opts(tools=['tap'])

In [ ]:
lonlat_stream = streams.Selection1D(source=latlon_table)

In [ ]:
lonlat_stream.contents

In [ ]:
def get_vectiles_from_inds(index, z=14):
    """
    inds: table row index list
    """
    overlay = []
    if not index: #no selection 
        return hv.Overlay([])

    for i in tqdm(index):
        lat, lon = latlon_table['lat'][i], latlon_table['lng'][i]
        country, city = latlon_table['country'][i], latlon_table['city'][i]
        overlay.append(get_vectile_overlay_at_latlon(lat,lon,z))
    return hv.Overlay(overlay)
def get_vectile_overlay_from_index(index, z=14):
    """
    index: table row index (int)
    
    """
    index = np.clip(index, 0, len(latlon_table)-1)
    lat, lon = latlon_table['lat'][index], latlon_table['lng'][index]
    country, city = latlon_table['country'][index], latlon_table['city'][index]
    return get_vectile_overlay_at_latlon(lat,lon,z)
                       
    

In [ ]:
# @output.capture()
(
    hv.DynamicMap(get_vectile_overlay_from_index, kdims='index')
    .redim.range(index=(0,len(latlon_table)-1))
).opts(width=600, height=600)

In [ ]:
vectiles = get_vectile_overlay_from_inds(list(range(130)), z=16)

In [ ]:
out = iw.Output(layout={'border': '1px solid black'})
out

# Continue from here
Modified: Jun 8, 2019

### Generate a varname dropdown box
Modified: Jun 3, 2019

In [ ]:
%%opts Image [width=600, height=600]
var_tmap = gv.Dataset(xrd_ea, kdims=kdims, vdims=varname, crs=ccrs.PlateCarree()).to(gv.Image, kdims=['X','Y'], vdims=varname)
var_tmap.redim(X='Lon', Y='Lat')

In [ ]:
def get_var_tmap(xr_dataset, varname, label=None, cmap=None):
    if label is None:
        label = varname
    tmap = gv.Dataset(xr_dataset, kdims=kdims, vdims=varname).to(gv.Image, kdims=['X','Y'], vdims=varname, label=label).redim(X='Lon', Y='Lat')
    print('type of tmap before datashade: ', type(tmap))
    print('\t\t after datashade: ', type(tmap))
    tmap = datashade(tmap, cmap=cmap)
    return tmap

def get_var_timg(xr_dataset, varname, t_idx, label=None, cmap=None):
    if label is None:
        label = varname  
    timg = gv.Dataset(xr_dataset.isel(time=t_idx), kdims=['X','Y'], vdims=varname).to(gv.Image, kdims=['X','Y'], vdims=varname, label=label).redim(X='Lon', Y='Lat')
    print('type of timg before datashade: ', type(timg))
    print('\t\t after datashade: ', type(timg))
    timg = datashade(timg, cmap=cmap)
    return timg


def test_get_timg():
    varname = varnames[1]
    t_idx = 1
    from matplotlib.cm import viridis

    timg = get_var_timg(xrd_ea, varname, t_idx, cmap=viridis)
    
    print('varname: ', varname, 't_idx: ', t_idx)
    display(timg)
    

In [ ]:
%%opts Image [width=600, height=600, tools=['hover']]
test_get_timg()

Let's add a base landscape vector features


In [ ]:
# %%opts Image [global_extent=True, height=600, width=600] (alpha=0.7) Image.EA (cmap='kbc') Image.SA (cmap='fire')
sq_cmaps = list_cmaps(provider='colorcet', category='Sequential')
varname = varnames[1]
t_idx = 1
timg_ea = get_var_timg(xrd_ea, varname, t_idx, label='EA', cmap=c_cm.kbc)
timg_sa = get_var_timg(xrd_sa, varname, t_idx, label='SA', cmap=c_cm.fire)



In [ ]:
overlay = timg_ea + timg_sa
print(overlay)

In [ ]:
# Let's see the current plot.state of timg_ea andn timg_sa
plot_ea = renderer.get_plot(timg_ea)
plot_sa = renderer.get_plot(timg_sa)

In [ ]:
def print_plot_state(hvElem, renderer):
    plot = renderer.get_plot(hvElem)
    print("Plot state: ")
    plot.print_param_values()
    
def print_param_state(hvElem):
    print("param values: ")
    hvElem.print_param_values()
    
def print_all_states(hvElem, renderer):
    print("="*80)
    print_param_state(hvElem)
    
    print("="*80)
    print_plot_state(hvElem, renderer)
    
def test_print_param_state():
    print_param_state(timg_sa)
def test_print_plot_state():
    print_plot_state( timg_sa, renderer)
def test_print_all_states():
    print_all_states(timg_sa, renderer)   
    
# test_print_param_state()
# test_print_plot_state()
# test_print_all_states()


In [ ]:
tmap_ea = get_var_tmap(xrd_ea, varname, label='EA', cmap=c_cm.kbc)
tmap_sa = get_var_tmap(xrd_sa, varname, label='SA', cmap=c_cm.fire)

In [ ]:
%%opts RGB [global_extent=False, height=500, width=500, tools=['hover']] (alpha=0.7) 
temp = (tmap_ea + tmap_sa).opts(width=500, height=500)
temp


In [ ]:
# Get tiles
tile = gvts.EsriImagery

In [ ]:
%%opts RGB [global_extent=False, height=H_IMG, width=W_IMG, tools=['hover']] (alpha=0.7) 
tile * tmap_ea * tmap_sa * gf.coastline

---
## TODO: 
Modified: Jun 4, 2019
- [ ] add latlon PointXY stream -> plot across time for this particular variable
- [ ] datashader statistics on a selected region by adding an extra layer on top of this:)
- [ ] Work with osm and spacenet dataset

In [ ]:
lon,lat = (22.35, -11.45)
pt = gv.Points([ (lon,lat) ], ['X', 'Y'])
LonLat = Stream.define('LonLat', X=22.35, Y=-11.45)

pt_stream = LonLat()
print(pt_stream.X, pt_stream.Y)

In [ ]:
tmap_ea * pt.opts(size=10, color='r')

In [ ]:
# PointerXY, a Linked Stream, to get a 2D dataview across Lat or Lon at the specific location and time
# We can explicitly add a source to the stream by passing the source object (eg. holompas, image) as an 
# argument to 'source' at the construction time of the Linked Stream object
pointer = streams.PointerXY(source=tmap_ea)
print(pointer.source is tmap_ea)

In [ ]:
# Let's see if it works
hv.DynamicMap(lambda x,y: hv.Points([(x,y)]).opts(size=10, framewise=True), streams=[pointer])

Awesome!


We can add the pointer stream to its original plot source (ie. `tmap_ea`)
def tmap_and_pointer(

In [ ]:
tmap_ea.redim.values(time=pd.date_range('2019-04-13', '2019-04-20'))

In [ ]:
hv.Dataset(xrd_ea.sel(X=lon, Y=lat, method='nearest'), 'time')

In [ ]:
def get_tseries_at_lon_lat(lon, lat):
    return (
        hv.Curve(xrd_ea.sel(X=lon, Y=lat, method='nearest'), 'time', varname)
        .relabel(f'Lon: {lon}, Lat: {lat}')
        .opts(width=1000, framewise=True)
    )


In [ ]:
def test_get_tseries_at_lon_lat():
    lon, lat = 35, -5
    display(get_tseries_at_lon_lat(lon, lat))
test_get_tseries_at_lon_lat()

In [ ]:
dmap = hv.DynamicMap(get_tseries_at_lon_lat, streams = [pointer.rename(x='lon', y='lat')])

In [ ]:
dmap.streams[0].source is pointer.source


In [ ]:
tmap_sa

In [ ]:
tmap_ea + dmap.opts(xticks=30)

The timeseries at a specific lat,lon is working! nice:)

## Basemap + EA, SA variable data + timeseries at lon/lat 
Modified: Jun 6, 2019

Let's put these together. Then we will 
- [ ] add another interactivity to select an area on the timage map and compute some meaning statistics on that region 
    - [ ] Use datashader's functionality here?
    
- [ ] take a look at the osm vector tile data (on the 2D space) 
- [ ] compute infrastructure distribution of 
    - [ ] the entire area
    - [ ] selected area
    
---
## Extra todos:
Modified: Jun 7, 2019
- [ ] if any of the coordinate of the pointer stream is out of bound, all tseries values should be zeros

 

1. Putting things together

In [ ]:
(
    (tile * tmap_ea).opts(width=1000,height=700) + \
    dmap.opts(xticks=30)
).cols(1)

In [ ]:
def get_tseries(xrd, varname, lon,lat):
    tseries = xrd[varname].sel(X=lon, Y=lat, method='nearest')
    return gv.Dataset(tseries, 'time', varname).to(hv.Curve)


In [ ]:
renamed_stream = LonLat().rename(X='lon', Y='lat')
# dmap_tseries = hv.DynamicMap(get_tseries, streams=[pt_stream.rename(X='lon', Y='lat')])
dmap_tseries = hv.DynamicMap(lambda lon,lat: get_tseries(xrd_ea, varnames[1], lon, lat), 
                            streams=[renamed_stream])

In [ ]:
dmap_tseries.opts(width=800, height=500)

In [ ]:
lons, lats, times = map(np.array, ds_ea[varname].coords.values())

In [ ]:

for i,(lon,lat) in enumerate(zip(lons[::10],lats[::10])):
    if i>9:
        break
    print(i, "lon,lat: ", lon,lat)
    dmap_tseries.event(lon=lon, lat=lat)
    time.sleep(3)
    
        

In [ ]:
dmap_tseries

In [ ]:

for i,(lon,lat) in enumerate(zip(lons[::10],lats[::10])):
    if i>5:
        break
    print(i, "lon,lat: ", lon,lat)
    dmap_tseries.event(lon=lon, lat=lat)
    time.sleep(1)
    

In [ ]:
dmap_tseries.opts(width=1000,height=300,framewise=True)

---
## Vector Tile Sources
Modified: Jun 5, 2019

In [ ]:
tile_url ='https://tile.nextzen.org/tilezen/vector/v1/512/all/16/19293/24641.json?api_key=GpjLSbvrQsa98MgMMuodzw'
gv.Tiles(tile_url, name="vector_tile").opts(width=600, height=550)


In [ ]:
gv.T

In [ ]:
vectile_path = '../data/24641.geojson'
gdf = gpd.read_file(vectile_path)
gdf.head()

In [ ]:
gdf.plot()

In [ ]:
gdf_vectile.crs


In [ ]:
import requests

In [ ]:
%%time
r = requests.get(url=tile_url)

In [ ]:
data = r.json()

In [ ]:
gdf2 = gpd.GeoDataFrame(data)

In [ ]:
gdf2

In [ ]:
import json
with open('./temp.geojson', 'w') as f:
    json.dump(data,f)

In [ ]:
gdf2 = gpd.read_file('./temp.geojson')
gdf2.head()

In [ ]:
#Modified: Jun 7, 2019
import json

VECTILE_CACHE = {}
def get_vectile_gdf_at_xyz(xtile, ytile, z,
                    size=256,layer='all',
                    fformat='json', 
                    cache_dir='../data/vectile_cache/'):
    """
    Given xtile, ytile and z(oom level), 
    request the vector tile from nextzen vector tile endpoint
    
    If the tile was requested before and is saved, 
    it will check the current python session's cache, then the local
    disk to read the tile from memory.
    
    If not cached, it will send a request to the vector tile server,
    save the tile data both in python memory and local disk.
    
    Returns geopandas.DataFrame that contains some meta data like osm_id 
    and most importantly) geometries
    
    Args:
    - xtile, ytile, z (int)
    - size (int) : currently only supports 256 because of the latlon->tile
        conversion calculation is constrained to that size
    - fformat (str): currently it must be json because I don't know
        how to read mvt or topojson formats to geopandas.DataFrame
        
    """
    #check if VEC_CACHE object exists in global namespace
    global VECTILE_CACHE
    try:
        VECTILE_CACHE
    except NameError:
        VECTILE_CACHE = {}
        
    cache_key = (size,layer,z,xtile,ytile)
    # Check if this tile is in python session memory
    # If so, read from the memory, otherwise read from the disk
    if VECTILE_CACHE.get( cache_key ):
        if VECTILE_CACHE[cache_key].get('loaded'):
            "Reading from python session memory"
            return VECTILE_CACHE[cache_key].get('dframe') #geopandas.gdf
        else:
            "Reading from disk cache"
            return gpd.GeoDataFrame.read_file(VECTILE_CACHE[cache_key].get('fpath'))
    
    # Request a new tile
    tile_url = f'https://tile.nextzen.org/tilezen/vector/v1/{size}/{layer}/{z}/{xtile}/{ytile}.{fformat}?api_key=GpjLSbvrQsa98MgMMuodzw'
    r = requests.get(url=tile_url)
    if not r.ok:
        raise ValueError('reponse not ok: ', r.status_code)
    data = r.json()
    
    # Write to disk
    fdir = (Path(cache_dir) / f'{size}/{layer}/{z}/{xtile}/').resolve()
    if not fdir.exists():
        fdir.mkdir(parents=True)
        print(f'{fdir} created')
    fpath = fdir/ f'{ytile}.{fformat}'
    print('Saving to: ', fpath)
    with open(fpath, 'w') as f:
        json.dump(data,f)
        
    while not fpath.exists():
        time.sleep(0.3)
    if fpath.exists():
        gdf = gpd.read_file(fpath)
    else:
        raise IOError('File was not correctly written to disk: ', fpath)
    
    # Write to cache
    VECTILE_CACHE[cache_key] = {
        'loaded': True,
        'dframe': gdf,
        'fpath': fpath
    }
    return gdf
def get_vectile_gdf_at_latlon(lat, lon, z,
                             **kwargs):
    """
    kwargs:
    - size (256) : tile size
    - layer ('all')
    - format='json', 
    - cache_dir='../data/vectile_cache/'
    """
    
    xtile, ytile = VectorTile.deg2tile_xy(lat, lon, z)

    return get_vectile_gdf_at_xyz(xtile, ytile, z, **kwargs)

def get_vectile_overlay_at_xyz(xtile, ytile, z, **kwargs):
    """
    Fetches the vector tile (from python cache or from the local disk or from the web service <- search order)
    and returns a NdOverlay of Shape Elements with a numeric index
    
    args:
    - xtile, ytile, z (int)

    kwargs:    
    - colors (iterable of color values, or str indicating a colormap definted in holoviews)
        : Used to generate a itertools.cycle to cycle through color values. 
        : Default is 'Category20'
        eg: color=bokeh.palettes.Category20_10
    - size: (default is 256) 
    - layer: (default is 'all')
    - fformat: (default is 'json')  
    - cache_dir: (default is '../data/vectile_cache/')
    
    For hv.Cycle('colomap_name')'s usage, refer to: 
    http://holoviews.org/user_guide/Style_Mapping.html
    """

    gdf = get_vectile_gdf_at_xyz(xtile, ytile, z, **kwargs)
    
    # colormap iterator
    import itertools
    from bokeh.palettes import Category20_10

    colors = kwargs.get('colors', 'Category20')
    
    # return ndoverlay of each shape
#     return hv.NdOverlay( {i:gv.Shape(geom).opts(fill_color=c) for i, (geom,c) in enumerate( zip(vtile_gdf.geometry, cmap_cycler) ) })
    return hv.NdOverlay({i: gv.Shape(geom).opts(fill_color=hv.Cycle(colors))
                      for (i,geom) in enumerate(gdf.geometry)})

def get_vectile_overlay_at_latlon(lat, lon, z, **kwargs):
    """
    Args:
    - lat, lon (float): lat lon in degrees
    - z (int): zoom level
    
    kwargs:    
    - colors (iterable): to be used to generate a itertools.cycle to cycle through 
        color values
        eg: color=bokeh.palettes.Category20_10
    - size: (default is 256) 
    - layer: (default is 'all')
    - fformat: (default is 'json')  
    - cache_dir: (default is '../data/vectile_cache/')
    """
    xtile, ytile = VectorTile.deg2tile_xy(lat, lon,z)

    return get_vectile_overlay_at_xyz(xtile, ytile, z, **kwargs)  


## Tests
def test_get_vectile_gdf():
    test_vectile = get_vectile_gdf()
#     print(test_vectile.geometry[0])
#     shapes = {i:gv.Shape(geom) for i,geom in enumerate(test_vectile.geometry)}
    vectiles = hv.NdOverlay({i:gv.Shape(geom) for i,geom in enumerate(test_vectile.geometry)})
    
    display(vectiles.opts(
        opts.Shape(cmap='viridis'))
           )
    

def test_get_vectile_overlay_at_xyz():
    xtile=19293
    ytile=24641
    z=16
    display(get_vectile_overlay_at_xyz(xtile, ytile,z))
    
    
def test_get_vectile_overlay_at_latlon():
#     lat, lon = (40.709792012434946, -74.0203857421875)
    lat, lon = (-10, 30)
    z=16
    display(get_vectile_overlay_at_latlon(lat, lon, z))
    
    
    

In [ ]:
test_get_vectile_overlay_at_latlon()

In [ ]:
lat, lon = (-10, 30)
z=16
display(get_vectile_overlay_at_latlon(lat, lon, z))

In [ ]:
VectorTile.deg2tile_xy(lat,lon,z)

In [ ]:
vtile_gdf = get_vectile_gdf()
vtile_gdf

In [ ]:
# Create colormap cycle iterator
import itertools
from bokeh.palettes import Category20_10
cmap_cycler = itertools.cycle(Category20_10)

In [ ]:
hv.NdOverlay( {i:gv.Shape(geom).opts(fill_color=c) for i, (geom,c) in enumerate( zip(vtile_gdf.geometry, cmap_cycler) ) })

In [ ]:
g = vtile_gdf.geometry[0]
display(gv.Shape(g).opts(fill_color='r'))
print(g)

In [ ]:
import math

class VectorTile():

    @staticmethod
    def deg2tile_xy(lat_deg, lon_deg, zoom):
        """
        Lat,Lon to tile numbers
        - src: https://is.gd/mjvdR7
        """
        lat_rad = math.radians(lat_deg)
        n = 2.0 ** zoom
        xtile = int((lon_deg + 180.0) / 360.0 * n)
        ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
        return (xtile, ytile)

    @staticmethod
    def tile_xyz2deg(xtile, ytile, zoom):
        """
        Tile numbers to lat/lon in degree
        This returns the NW-corner of the square. 
        Use the function with xtile+1 and/or ytile+1 to get the other corners. 
        With xtile+0.5 & ytile+0.5 it will return the center of the tile.
        - src: https://is.gd/mjvdR7
        """
        n = 2.0 ** zoom
        lon_deg = xtile / n * 360.0 - 180.0
        lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
        lat_deg = math.degrees(lat_rad)
        return (lat_deg, lon_deg)

In [ ]:
VectorTile.tile_xyz2deg(19293, 24641, 16)

In [ ]:
vectiles = hv.NdOverlay({i:gv.Shape(geom) for i,geom in enumerate(test.geometry)})

In [ ]:
vtile_ndo = get_vectile_overlay(

In [ ]:
f

Let's get a better vector features with national boundaries. `geoviews.features` is a warpper of `cartopy.crs.feature` which supports easy feature collection downloading from `Natural Earth Data`.
    - [link](https://is.gd/jmmPRV)


In [ ]:
gf_land_110 = gv.Feature(cf.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '110m'))

# nice bboxfinder
# https://is.gd/L163zr
extents = (-32.343750,-43.580391,69.257813,47.279229)
xlim = (extents[0], extents[2])
ylim = (extents[1], extents[3])

# reset the extent for visualization only
# gf_land_110.extents =  extents
# gf_land_110

In [ ]:
timg_ea.bounds

In [ ]:
gf_land_110.extents

In [ ]:
%%opts Feature [global_extent=False]
gf_land_110

In [ ]:
%%opts WMTS [height=1200, width=1200] Image [height=1200, width=1200] (alpha=0.7) Image.EA (cmap='kbc') Image.SA (cmap='fire')
tile * gf_land_110 * tmap_ea * tmap_sa

In [ ]:
%%opts  Image [ height=600, width=600] (alpha=0.7) Image.EA (cmap='kbc') Image.SA (cmap='fire')

# gf_land_110 * timg_ea

In [ ]:
gf_land_110.extents=tuple([None for i in range(4)])

In [ ]:
gf_land_110

In [ ]:
timg_ea.opts(opts.Image(xlim=xlim, ylim=ylim))

In [ ]:
(timg_ea * timg_sa * gf_land_110)


In [ ]:


# ( 
#     (timg_ea * gf_land_110) + \
#     (timg_sa * gf_land_110) + \
#     (timg_ea * timg_sa * gf_land_110)
# ).cols(1)

In [ ]:
%%opts Feature [global_extent=True, height=600, width=1200] Points [global_extent=True] (size=10)
# WMTS [projection=ccrs.Geostationary()] Image [projection=ccrs.Geostationary()]
lon, lat = 40.0, 8.0
pt = gv.Points([(lon,lat)], crs=ccrs.PlateCarree())
(gf.land * pt)

---
## Load vector data using geopandas interface
Modified: Jun 4, 2019

In [ ]:
fpath = '../assets/cities.csv'
gdf = gpd.read_file(fpath)
gdf.head()

---
## Load dataset using intake

In [ ]:
cat = intake.open_catalog('./fldas_catalog.yml')
list(cat)

In [ ]:
ds_ea1 = cat.fldas_noah01_a_regional_daily_single(region='EA', day=2)
xr_ea1 = ds_ea1.to_dask()

In [ ]:
xr_ea1

In [ ]:
ds_ea = cat.fldas_noah01_a_regional_daily_all(region='EA')
ds_sa = cat.fldas_noah01_a_regional_daily_all(region='SA')

In [ ]:
%%time
ea = ds_ea.read()

In [ ]:
%%time
sa = ds_sa.read()

In [ ]:
ea = ea.drop_dims('bnds')
sa = sa.drop_dims('bnds')

In [ ]:
ea


In [ ]:
kdims = ['X', 'Y', 'time']
varnames = list(ea.data_vars.keys())
print(varnames)



In [ ]:
hvd_ea = hv.Dataset(ea, kdims=kdims)
hvd_sa = hv.Dataset(sa, kdims=kdims)
# print(hvd_ea)

In [ ]:
def get_hvm_var(xr_dset, varname):
    
    hvd_var = hv.Dataset(xr_dset[varname], kdims=kdims)
    hvd_var.to(gv.Image, ['X','Y'], dynamic=True)
    
get_hvm_var(hvd_sa, varnames[2])

In [ ]:
def get_varimg(xrd, varname, t):
    hvd = hv.Dataset(xrd[varname].isel(time=t), kdims=['X','Y'])
    return hvd.to(gv.Image, kdims=['X','Y'], vdims=varname)

varname = varnames[3]
t = 19
img0 = get_varimg(ea, varname, t)

In [ ]:
dmap = hv.DynamicMap(lambda varname, t: get_varimg(ea, varname, t),
            kdims=['varname', 't'])
              

In [ ]:
dmap[varnames[-1,1]

In [ ]:
varname = varnames[1]
hv.Dataset(ea[varname], kdims=['X','Y','time']).to(gv.Image, ['X','Y'], dynamic=True)

In [ ]:
tile = gv.tile_sources.EsriImagery
tile

In [ ]:
%%opts Overlay [global_extent=True, width=600, height=600]
gf.coastline * img0

In [ ]:
img0